In [186]:
# считываение из файла
def read_input(file_str: str):
    with open(file_str) as f:

        lines = f.readlines()
        lines.reverse()

        n = int(lines.pop().split()[0])
        parents = [int(s) for s in lines.pop().split(' ')]
        values = [int(s) for s in lines.pop().split(' ')]

        diseases = []
        num_of_diseases = int(lines.pop().split()[0])
        for i in range(num_of_diseases):
            dis = lines.pop()
            diseases.append([int(s) for s in dis.split(' ')])

        patients = []
        num_of_patients = int(lines.pop().split()[0])
        for i in range(num_of_patients):
            dis = lines.pop()
            patients.append([int(s) for s in dis.split(' ')])

        # return [n, relationships, values, diseases, patients]
        return {
            "n": n,
            "parents": parents,
            "values": values,
            "diseases": diseases,
            "patients": patients,
        }

data = read_input('input.txt')
n = data["n"]
parents = data["parents"]
values = data["values"]
diseases = data["diseases"]
patients = data["patients"]

In [187]:
from collections import defaultdict

parents_hmap = defaultdict(int)  # child: parent
children_hmap = defaultdict(list)  # parent: [children]
values_hmap = defaultdict(int)  # node: value
is_checked_hmap = defaultdict(bool)  # node: value

values_hmap[1] = values[0]
is_checked_hmap[1] = False
for i in range(len(parents)):
    child_index = i + 2
    parent_index = parents[i]

    parents_hmap[child_index] = parent_index
    children_hmap[parent_index].append(child_index)
    values_hmap[child_index] = values[i + 1]
    is_checked_hmap[child_index] = False

print(parents_hmap)
print(children_hmap)
print(values_hmap)
print(is_checked_hmap)

defaultdict(<class 'int'>, {2: 1, 3: 1, 4: 3, 5: 3, 6: 4, 7: 4, 8: 5, 9: 5, 10: 5})
defaultdict(<class 'list'>, {1: [2, 3], 3: [4, 5], 4: [6, 7], 5: [8, 9, 10]})
defaultdict(<class 'int'>, {1: 5, 2: 7, 3: 8, 4: 13, 5: 18, 6: 14, 7: 15, 8: 21, 9: 20, 10: 29})
defaultdict(<class 'bool'>, {1: False, 2: False, 3: False, 4: False, 5: False, 6: False, 7: False, 8: False, 9: False, 10: False})


In [197]:
subtrree_hmap = defaultdict(list)  # parent: [all children in subtree]

def fill_subtrees(cur):
    children = children_hmap[cur]
    subtrree_hmap[cur].append(cur)
    if len(children) == 0:
        return [cur]

    for child in children:
        for el in fill_subtrees(child):
            subtrree_hmap[cur].append(el)
    return subtrree_hmap[cur]

fill_subtrees(1)
print(subtrree_hmap)

defaultdict(<class 'list'>, {1: [1, 2, 3, 4, 6, 7, 5, 8, 9, 10], 2: [2], 3: [3, 4, 6, 7, 5, 8, 9, 10], 4: [4, 6, 7], 6: [6], 7: [7], 5: [5, 8, 9, 10], 8: [8], 9: [9], 10: [10]})


In [207]:
common_parent_hmap = defaultdict()

def fill_common_parent_hmap(cur):
    subtree = subtrree_hmap[cur]
    for el in subtree:
        common_parent_hmap[cur, el] = values_hmap[cur]
        common_parent_hmap[el, cur] = values_hmap[cur]
    children = children_hmap[cur]
    for child in children:
        fill_common_parent_hmap(child)

fill_common_parent_hmap(1)
print(common_parent_hmap)

defaultdict(None, {(1, 1): 5, (1, 2): 5, (2, 1): 5, (1, 3): 5, (3, 1): 5, (1, 4): 5, (4, 1): 5, (1, 6): 5, (6, 1): 5, (1, 7): 5, (7, 1): 5, (1, 5): 5, (5, 1): 5, (1, 8): 5, (8, 1): 5, (1, 9): 5, (9, 1): 5, (1, 10): 5, (10, 1): 5, (2, 2): 7, (3, 3): 8, (3, 4): 8, (4, 3): 8, (3, 6): 8, (6, 3): 8, (3, 7): 8, (7, 3): 8, (3, 5): 8, (5, 3): 8, (3, 8): 8, (8, 3): 8, (3, 9): 8, (9, 3): 8, (3, 10): 8, (10, 3): 8, (4, 4): 13, (4, 6): 13, (6, 4): 13, (4, 7): 13, (7, 4): 13, (6, 6): 14, (7, 7): 15, (5, 5): 18, (5, 8): 18, (8, 5): 18, (5, 9): 18, (9, 5): 18, (5, 10): 18, (10, 5): 18, (8, 8): 21, (9, 9): 20, (10, 10): 29})


In [206]:
for patient in patients:
    max_p = 0
    for disease in diseases:
        sum = 0
        for p in disease:
            max_d = 0
            for d in disease:
                IC_LCA = common_parent_hmap[p, d]
                if IC_LCA > max_d:
                    max_d = IC_LCA
            sum += max_d
        if sum > max_p:
                max_p = sum
    print(max_p)

KeyError: (2, 4)